In [1]:
import pandas as pd  
import numpy as np
import talib as ta
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode

In [2]:
HOUR = pd.read_csv("btc_hour.csv")
DAY = pd.read_csv("btc_day.csv")

In [3]:
#isnan
HOUR.isnull().values.any()
DAY.isnull().values.any()

False

In [4]:
def refined(df):
    df['up']=ta.MAX(df.high,timeperiod=20).shift(1)
    #最近N2个交易日最低价
    df['down']=ta.MIN(df.low,timeperiod=10).shift(1)
    #每日真实波动幅度
    df['ATR']=ta.ATR(df.high,df.low,df.close,timeperiod=20)
    df['ret']=df.close/df.close.shift(1)-1
    df.loc[0,'ret'] = 0
    df['signal'] = 0
    #超上限，跌下限
    df.signal[df.close>df.up] = 1
    df.signal[df.close<df.down] = -1
    return df

In [37]:
#realize in or out, add or stop
def calunit(df,account):
    df['unit'] = 0
    status = 'null'
    price = 0
    amount = 0
    df.loc[0,'capital'] = account
    for i in range(1,len(df)):
        if df.signal[i] == 1 and status == 'null':
            C = df.loc[i-1,'capital']
            df.loc[i,'unit'] = int((1/df.ATR[i])*C/100)
            status = 'hold'
            price = df.close[i]
            amount = df.loc[i,'unit']*price + amount
            df.loc[i,'capital'] = df.loc[i,'unit']*df.loc[i,'close']+(C-amount)
        elif df.signal[i] != -1 and status == 'hold':
            if df.close[i] >= price + 0.5*df.ATR[i]:
                if int((1/df.ATR[i])/100*C)*df.loc[i,'close']<(C-amount):
                    df.loc[i,'unit'] = int((1/df.ATR[i])/100*C)+df.loc[i-1,'unit']
                    price = df.close[i]
                    amount = (df.loc[i,'unit']-df.loc[i-1,'unit'])*df.close[i] + amount
                    df.loc[i,'capital'] = df.loc[i,'unit']*df.loc[i,'close']+(C-amount)
                else:
                    df.loc[i,'unit'] = int((C-amount)/df.loc[i,'close'])+df.loc[i-1,'unit']
                    price = df.close[i]
                    amount = (df.loc[i,'unit']-df.loc[i-1,'unit'])*df.close[i] + amount
                    df.loc[i,'capital'] = df.loc[i,'unit']*df.loc[i,'close']+(C-amount)
            elif df.close[i] <= price - 2*df.ATR[i]:
                df.loc[i,'unit'] = 0
                if df.loc[i-1,'unit'] == 0:
                    df.loc[i,'capital'] = df.loc[i-1,'capital']
                else:
                    df.loc[i,'capital'] = df.loc[i-1,'unit']*df.loc[i,'close']+(C-amount)
                amount = 0
                C = df.loc[i-1,'capital']
            else:
                df.loc[i,'unit'] = df.loc[i-1,'unit']
                if df.loc[i,'unit'] == 0:
                    df.loc[i,'capital'] = df.loc[i-1,'capital']
                else:
                    df.loc[i,'capital'] = df.loc[i,'unit']*df.loc[i,'close']+(C-amount)
        elif df.signal[i] == 0 and status == 'null':
            df.loc[i,'capital'] = df.loc[i-1,'capital']
        elif df.signal[i] == -1:
            df.loc[i,'unit'] = 0
            status = 'null'
            if df.loc[i-1,'unit'] == 0:
                df.loc[i,'capital'] = df.loc[i-1,'capital']
            else:
                df.loc[i,'capital'] = df.loc[i-1,'unit']*df.loc[i,'close']+(C-amount)
            amount = 0
            
    df['record'] = df['unit'].diff()
    df.loc[0,'record'] = 0
    return df

In [93]:
re_day = refined(DAY)

C:\Users\abbya\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\abbya\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


以下是因为还是要按照每日的UP,DOWN,ATR值来判断加仓减仓，而不是每小时

In [68]:
re_hour = pd.merge(HOUR,re_day[['day','up','down','ATR']],on='day',how='left')

In [69]:
re_hour['ret']=re_hour.close/re_hour.close.shift(1)-1
re_hour.loc[0,'ret'] = 0
re_hour['signal'] = 0
re_hour.signal[re_hour.close>re_hour.up] = 1
re_hour.signal[re_hour.close<re_hour.down] = -1
re_hour

C:\Users\abbya\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\abbya\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Unnamed: 0,time,open,high,low,close,volume,day,hour,up,down,ATR,ret,signal
0,0,1502942400000,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17,2017-08-17 04:00,NaN,NaN,NaN,0.000000,0
1,1,1502946000000,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17,2017-08-17 05:00,NaN,NaN,NaN,0.001506,0
2,2,1502949600000,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17,2017-08-17 06:00,NaN,NaN,NaN,0.002093,0
3,3,1502953200000,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17,2017-08-17 07:00,NaN,NaN,NaN,0.005929,0
4,4,1502956800000,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17,2017-08-17 08:00,NaN,NaN,NaN,0.002460,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31613,31613,1617217200000,59304.45,59567.28,58453.30,58553.12,3276.305903,2021-03-31,2021-03-31 19:00,61844.0,50427.56,3548.128964,-0.012586,0
31614,31614,1617220800000,58553.12,59087.00,58328.55,58934.60,3283.190916,2021-03-31,2021-03-31 20:00,61844.0,50427.56,3548.128964,0.006515,0
31615,31615,1617224400000,58935.10,59289.45,58700.00,58940.50,1671.199727,2021-03-31,2021-03-31 21:00,61844.0,50427.56,3548.128964,0.000100,0
31616,31616,1617228000000,58940.50,59009.85,58558.00,58621.94,1447.622712,2021-03-31,2021-03-31 22:00,61844.0,50427.56,3548.128964,-0.005405,0


In [95]:
account = 1000000   #因为BTC最高价大于5万，所以这里选择初始资金为10万，可自行修改
#re_day = calunit(re_day,account)
re_hour = calunit(re_hour,account)

In [44]:
#re_hour[['signal','unit','record','capital']].describe()
#re_hour.to_csv("rehour.csv")
#re_day.to_csv("reday.csv")

In [35]:
#np.std(re_hour['close'].diff(), ddof = 1)

至此得到了，含加仓减仓等信息的重组数据

# 指标计算

In [107]:
df = re_hour.copy()
#df = re_day.copy() 

In [108]:
#df['strategyNet']=(df.close.diff()*df.unit).cumsum()
df['holdingNet']=(df.close.diff()*int(account/df.close[0])).cumsum()
df['strategyRate'] = (df['capital'].diff()+abs(df['record'])*df['close'].shift(1))/(df['capital'].shift(1)+abs(df['record'])*df['close'].shift(1))
df['holdingRate']=df['holdingNet']/df['holdingNet'].shift(1)-1
df.loc[np.isinf(df['strategyRate']),'strategyRate'] = np.nan
df['strategyCumRate']=df['capital']/account-1
df['holdingCumRate']=df['holdingNet']/account-1

In [109]:
#df[['strategyNet','holdingNet','strategyRate','holdingRate']].describe()
df['unit'].describe()
df[df['signal']==1]

,Unnamed: 0,time,open,high,low,close,volume,day,hour,up,...,ret,signal,unit,capital,record,holdingNet,strategyRate,holdingRate,strategyCumRate,holdingCumRate
1092,1092,1506902400000,4400.00,4430.00,4380.01,4430.00,17.485013,2017-10-02,2017-10-02 00:00,4406.52,...,0.011767,1,30,1000000.00,30.0,28111.44,0.116104,0.739698,0.000000,-0.971889
1093,1093,1506906000000,4420.00,4560.00,4404.49,4437.98,55.771295,2017-10-02,2017-10-02 01:00,4406.52,...,0.001801,1,30,1000239.40,0.0,29962.80,0.000239,0.065858,0.000239,-0.970037
1094,1094,1506909600000,4410.01,4482.72,4409.01,4419.99,28.921834,2017-10-02,2017-10-02 02:00,4406.52,...,-0.004054,1,30,999699.70,0.0,25789.12,-0.000540,-0.139295,-0.000300,-0.974211
1095,1095,1506913200000,4414.05,4440.00,4390.00,4419.85,25.470699,2017-10-02,2017-10-02 03:00,4406.52,...,-0.000032,1,30,999695.50,0.0,25756.64,-0.000004,-0.001259,-0.000305,-0.974243
1096,1096,1506916800000,4411.00,4459.82,4405.15,4427.95,17.165915,2017-10-02,2017-10-02 04:00,4406.52,...,0.001833,1,30,999938.50,0.0,27635.84,0.000243,0.072960,-0.000062,-0.972364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31181,31181,1615662000000,60281.95,60864.97,60144.20,60773.56,3093.953168,2021-03-13,2021-03-13 19:00,58352.80,...,0.008155,1,7,2887861.32,0.0,13099817.36,0.001193,0.008783,1.887861,12.099817
31182,31182,1615665600000,60773.56,61844.00,60651.74,61648.26,6042.502819,2021-03-13,2021-03-13 20:00,58352.80,...,0.014393,1,14,2893984.22,7.0,13302747.76,0.130245,0.015491,1.893984,12.302748
31183,31183,1615669200000,61648.26,61737.02,60605.50,61151.07,4486.032217,2021-03-13,2021-03-13 21:00,58352.80,...,-0.008065,1,14,2887023.56,0.0,13187399.68,-0.002405,-0.008671,1.887024,12.187400
31184,31184,1615672800000,61156.40,61311.19,60767.62,61096.04,1770.676941,2021-03-13,2021-03-13 22:00,58352.80,...,-0.000900,1,14,2886253.14,0.0,13174632.72,-0.000267,-0.000968,1.886253,12.174633


In [110]:
annual_ret=pow(df[['capital','holdingNet']].iloc[-1]/account,250/len(DAY))-1
annual_ret

capital       0.213163
holdingNet    0.614783
Name: 31617, dtype: float64

In [111]:
beta=df[['strategyRate','holdingRate']].cov().iat[0,1]/(df['holdingRate'].var())
beta

0.00010196686360007053

In [112]:
alpha=(annual_ret['capital']-annual_ret['holdingNet']*beta)
alpha

0.2131007710917493

In [113]:
vol =  np.std(df['strategyRate'], ddof = 1)
vol

0.01709053485015197

In [114]:
sharperR = annual_ret['capital']/vol
sharperR

12.472603136484155

In [115]:
inforR = (annual_ret['capital']-annual_ret['holdingNet'])/np.std(df['strategyRate']-df['holdingRate'], ddof = 1)
inforR

-0.6118970931243309

In [116]:
strategyMaxR = ((df['capital']-df['capital'].expanding().min())/(df['capital']+account)).max()
holdingMaxR = ((df['holdingNet']-df['holdingNet'].expanding().min())/(df['holdingNet']+account)).max()
[strategyMaxR,holdingMaxR]

[0.5480738275535916, 0.9526273236884658]

# 画图

In [117]:
#back test
(
    Line(init_opts=opts.InitOpts(width="1200px", height="600px"))
    .add_xaxis(xaxis_data=df.hour.tolist())  #daily数据要改为df.day.tolist()
    .add_yaxis(
        series_name="strategy Cumulative return",
        y_axis=round(df.strategyCumRate,2).tolist(),
        label_opts=opts.LabelOpts(is_show=False,formatter=JsCode("function (params) {return params.value[1] + '%'}")),
        
    )
    .add_yaxis(
        series_name="holding Cumulative return",
        y_axis=round(df.holdingCumRate,2).tolist(),
        label_opts=opts.LabelOpts(is_show=False,formatter=JsCode("function (params) {return params.value[1] + '%'}")),
        
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="BTC:backtest of turtle strategy", 
                                  subtitle=f'annualized return: strategy:{round(annual_ret.capital*100,3)}%, holding:{round(annual_ret.holdingNet*100,3)}%; maxR: strategy:{round(strategyMaxR*100,3)}%, holding:{round(holdingMaxR*100,3)}%;\n alpha:{round(alpha,3)}, beta:{round(beta,3)},shaperRatio:{round(sharperR,3)},information ratio: {inforR}' ,
                                  subtitle_textstyle_opts= opts.TextStyleOpts(color="black",font_size=14,font_weight='bold'),
                                 pos_top = 1,
                                 item_gap = 1,),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
    )
    .render("btc_hour_backtest.html")  #画新图要改名
)

'C:\\Users\\abbya\\5530 group\\btc_hour_backtest.html'

In [22]:
#kline,bar图还没弄清楚
bar = (
    Bar()
    .add_xaxis(df.day.tolist())
    .add_yaxis(
        "volume",
        df.volume.tolist(),
        color="#d14a61",
        label_opts=opts.LabelOpts(is_show=False),
    )
   .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            is_show = False,
            ),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        datazoom_opts=[opts.DataZoomOpts(type_="inside")],
    )
)
line = (
    Line()
    .add_xaxis(df.hour.tolist())
    .add_yaxis(
        "UP",
        df.up.tolist(),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="DOWN",
        y_axis=df.down.tolist(),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="BTC Overall"),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        datazoom_opts=[opts.DataZoomOpts(type_="inside")],
    )
)

kline = (
    Kline()
    .add_xaxis(df.hour.tolist())
    .add_yaxis(
        "kline",
        np.array(df.loc[:,['open','close','low','high']]).tolist(),
        itemstyle_opts=opts.ItemStyleOpts(
            color="#ec0000",
            color0="#00da3c",
            border_color="#8A0000",
            border_color0="#008F28",
        ),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(type_="inside")],
        title_opts=opts.TitleOpts(title="Kline-BTC"),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
    )
)

overlap = line.overlap(kline)
#overlap_1 = bar.overlap(line)
#overlap_2 = bar1.overlap(line1)

grid = (
    Grid(init_opts=opts.InitOpts(width="1200px", height="800px"))
    .add(
        overlap,grid_opts=opts.GridOpts(pos_bottom="30%"),is_control_axis_index=True
    )
    #.add(bar, grid_opts=opts.GridOpts(pos_top="80%"))
    .render("btc_hour_overall.html")
)

ref:
指标计算：https://uqer.datayes.com/help/faqApi/#%E6%94%B6%E7%9B%8A%E5%92%8C%E9%A3%8E%E9%99%A9%E6%8C%87%E6%A0%87
https://www.investopedia.com/terms/t/time-weightedror.asp